In [1]:
!pip install spektral

     -------------------------------------- 140.1/140.1 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 6.8 MB/s eta 0:00:00
     -------------------------------------- 895.9/895.9 kB 7.1 MB/s eta 0:00:00
     ---------------------------------------- 23.2/23.2 MB 2.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.1
    Uninstalling protobuf-3.20.1:
      Successfully uninstalled protobuf-3.20.1


In [2]:
!pip show spektral

Name: spektral
Version: 1.2.0
Summary: Graph Neural Networks with Keras and Tensorflow 2.
Home-page: https://github.com/danielegrattarola/spektral
Author: Daniele Grattarola
Author-email: daniele.grattarola@gmail.com
License: MIT
Location: c:\users\mohds\anaconda3\lib\site-packages
Requires: joblib, lxml, networkx, numpy, pandas, requests, scikit-learn, scipy, tensorflow, tqdm
Required-by: 


In [3]:
import numpy as np
import tensorflow as tf
import spektral

In [7]:
cora_dataset = spektral.datasets.citation.Citation(name='cora')
test_mask = cora_dataset.mask_te
train_mask = cora_dataset.mask_tr
val_mask = cora_dataset.mask_va
graph = cora_dataset.graphs[0]
features = graph.x
adj = graph.a
labels = graph.y

print(features.shape)
print(adj.shape)
print(labels.shape)

print(np.sum(train_mask))
print(np.sum(val_mask))
print(np.sum(test_mask))

(2708, 1433)
(2708, 2708)
(2708, 7)
140
500
1000


C:\Users\MohdS\anaconda3\lib\site-packages\spektral\datasets\citation.py:108: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  a = nx.adjacency_matrix(nx.from_dict_of_lists(graph))  # CSR
C:\Users\MohdS\anaconda3\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [8]:
def masked_softmax_cross_entropy(logits, labels, mask):
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    loss *= mask
    return tf.reduce_mean(loss)

def masked_accuracy(logits, labels, mask):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy_all = tf.cast(correct_prediction, tf.float32)
    mask = tf.cast(mask, dtype=tf.float32)
    mask /= tf.reduce_mean(mask)
    accuracy_all *= mask
    return tf.reduce_mean(accuracy_all)

In [12]:
def gnn(fts, adj, transform, activation):
    seq_fts = transform(fts)
    adj = adj.todense()
    ret_fts = tf.matmul(adj, seq_fts)
    return activation(ret_fts)

In [13]:
def train_cora(fts, adj, gnn_fn, units, epochs, lr):
    lyr_1 = tf.keras.layers.Dense(units)
    lyr_2 = tf.keras.layers.Dense(7)
    
    def cora_gnn(fts, adj):
        hidden = gnn_fn(fts, adj, lyr_1, tf.nn.relu)
        logits = gnn_fn(hidden, adj, lyr_2, tf.identity)
        return logits
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    
    best_accuracy = 0.0
    for ep in range(epochs + 1):
        with tf.GradientTape() as t:
            logits = cora_gnn(fts, adj)
            loss = masked_softmax_cross_entropy(logits, labels, train_mask)
            
        variables = t.watched_variables()
        grads = t.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))
        
        logits = cora_gnn(fts, adj)
        val_accuracy = masked_accuracy(logits, labels, val_mask)
        test_accuracy = masked_accuracy(logits, labels, test_mask)
        
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            print('Epoch', ep, '| Training loss:', loss.numpy(), 'Val accuracy:',
                  val_accuracy.numpy(), '|Test accuracy:', test_accuracy.numpy())

In [14]:
train_cora(features, adj, gnn, 32, 200, 0.01)

Epoch 0 | Training loss: 3.465349 Val accuracy: 0.284 |Test accuracy: 0.298
Epoch 1 | Training loss: 5.3994603 Val accuracy: 0.52599996 |Test accuracy: 0.545
Epoch 2 | Training loss: 2.0442512 Val accuracy: 0.626 |Test accuracy: 0.606
Epoch 4 | Training loss: 0.9207304 Val accuracy: 0.64 |Test accuracy: 0.624
Epoch 5 | Training loss: 0.82862926 Val accuracy: 0.666 |Test accuracy: 0.66499996
Epoch 6 | Training loss: 0.6962547 Val accuracy: 0.69600004 |Test accuracy: 0.695
Epoch 9 | Training loss: 0.5267703 Val accuracy: 0.74 |Test accuracy: 0.74399996
Epoch 10 | Training loss: 0.41576472 Val accuracy: 0.75200003 |Test accuracy: 0.75200003
